In [1]:
%matplotlib inline
import fceulib
from fceulib import VectorBytes
import numpy as np
import time
import matplotlib.pyplot as plt
from fceu_help import *
from math import *
import pickle

In [2]:

START = 0x08
RUN = 0x02
JUMP = 0x01

#inputVec = fceulib.readInputs('videos/algar-batman.fm2')
#emu = fceulib.runGame('batman.nes')

inputVec = fceulib.readInputs('Illustrative.fm2')#'videos/happylee4-smb-warpless.fm2')
emu = fceulib.runGame('mario.nes')

#inputVec = fceulib.readInputs('videos/Contra by 3D.fm2')
#emu = fceulib.runGame('contra.nes')

start = VectorBytes()
emu.save(start)



filename = 'Illustrative.txt'
fc = emu.fc


In [4]:
#uses normalized pmi as a thresholding mechanism 
# -1 = no co-occurences, 
# 0 = independent 
# 1 = only co-occur

threshold=0.1
#Read in data
timesteps = {}
pSprite = {}
spr = get_sprite(0,emu.fc)
height = spr.shape[0]
#for each timestep look at all the sprite pairs and see how far apart they
with open(filename) as infile:
    for line in infile:
        dat = [int(float(d)) for d in line.rstrip().split(',')]
        if dat[0] not in timesteps:
            timesteps[dat[0]] = []
            if dat[0] % 1000 == 0:
                print dat[0]
        timesteps[dat[0]].append(dat[1:])
        print dat
        if dat[1] not in pSprite:
            pSprite[dat[1]] = 0
        pSprite[dat[1]] += 1
        
        

[270, 0, 88, 25]
[270, 1, 40, 193]
[270, 2, 48, 193]
[270, 3, 40, 201]
[270, 4, 48, 201]
[271, 0, 88, 25]
[271, 1, 40, 193]
[271, 2, 48, 193]
[271, 3, 40, 201]
[271, 4, 48, 201]
[272, 0, 88, 25]
[272, 1, 40, 193]
[272, 2, 48, 193]
[272, 3, 40, 201]
[272, 4, 48, 201]
[273, 0, 88, 25]
[273, 1, 40, 193]
[273, 2, 48, 193]
[273, 3, 40, 201]
[273, 4, 48, 201]
[274, 0, 88, 25]
[274, 1, 40, 193]
[274, 2, 48, 193]
[274, 3, 40, 201]
[274, 4, 48, 201]
[275, 0, 88, 25]
[275, 1, 40, 193]
[275, 2, 48, 193]
[275, 3, 40, 201]
[275, 4, 48, 201]
[276, 0, 88, 25]
[276, 1, 40, 193]
[276, 2, 48, 193]
[276, 3, 40, 201]
[276, 4, 48, 201]
[277, 0, 88, 25]
[277, 1, 40, 193]
[277, 2, 48, 193]
[277, 3, 40, 201]
[277, 4, 48, 201]
[278, 0, 88, 25]
[278, 1, 40, 193]
[278, 2, 48, 193]
[278, 3, 40, 201]
[278, 4, 48, 201]
[279, 0, 88, 25]
[279, 5, 40, 193]
[279, 6, 48, 193]
[279, 7, 40, 201]
[279, 8, 48, 201]
[280, 0, 88, 25]
[280, 9, 40, 193]
[280, 2, 48, 193]
[280, 10, 40, 201]
[280, 11, 48, 201]
[281, 0, 88, 25]
[2

In [5]:
#get the per timestep data, as well as total counts for each sprite are,
#if they are within width and height of each other, add that as a co-occurence

all_sprites = {}
sorted_timesteps = sorted(timesteps)
last =  sorted_timesteps[-1]
for timestep in sorted_timesteps:
    dat = timesteps[timestep]
    if timestep% 1000 == 0:
        print timestep
    for ii in range(len(dat)):
        for jj in range(len(dat)):
            if ii != jj:
                id1 = dat[ii][0]
                dx = dat[ii][1]-dat[jj][1]
                dy = dat[ii][2]-dat[jj][2]

                id2 = (dat[jj][0], dx, dy)
                if (abs(dx) != 8 and abs(dx) != 0)  or (abs(dy) != height and abs(dy) != 0):
                    continue
                if id1 not in all_sprites:
                    all_sprites[id1] = {}
                if id2 not in all_sprites[id1]:
                    all_sprites[id1][id2] = 0
                all_sprites[id1][id2] += 1

1000
2000
3000
4000
5000
6000
7000


In [6]:

colorized2id,id2colorized = pickle.load(open('id2sprites.pkl','rb'))
#now we have all of the pieces and can calculate the pmi for each pair of sprites
accepted = set()
seen = set()
for sprite in all_sprites:
    total = 0
    dat = []
    px = float(pSprite[sprite])/float(len(timesteps))

    for other in all_sprites[sprite]:
        py = float(pSprite[other[0]])/float(len(timesteps))
        pxy = float(all_sprites[sprite][other])/float(len(timesteps))
        d = log(px*py)/log(pxy)-1
        if d > threshold:
            s = sprite
            o = other
            if sprite > other:
                o = sprite
                s = other
            accepted.add((s,o))

In [7]:
"""UnionFind.py

Union-find data structure. Based on Josiah Carlson's code,
http://aspn.activestate.com/ASPN/Cookbook/Python/Recipe/215912
with significant additional changes by D. Eppstein.
"""
class UnionFind:
    """Union-find data structure.

    Each unionFind instance X maintains a family of disjoint sets of
    hashable objects, supporting the following two methods:

    - X[item] returns a name for the set containing the given item.
      Each set is named by an arbitrarily-chosen one of its members; as
      long as the set remains unchanged it will keep the same name. If
      the item is not yet part of a set in X, a new singleton set is
      created for it.

    - X.union(item1, item2, ...) merges the sets containing each item
      into a single larger set.  If any item is not yet part of a set
      in X, it is added to X as one of the members of the merged set.
    """

    def __init__(self):
        """Create a new empty union-find structure."""
        self.weights = {}
        self.parents = {}

    def __getitem__(self, object):
        """Find and return the name of the set containing the object."""

        # check for previously unknown object
        if object not in self.parents:
            self.parents[object] = object
            self.weights[object] = 1
            return object

        # find path of objects leading to the root
        path = [object]
        root = self.parents[object]
        while root != path[-1]:
            path.append(root)
            root = self.parents[root]

        # compress the path and return
        for ancestor in path:
            self.parents[ancestor] = root
        return root
        
    def __iter__(self):
        """Iterate through all items ever found or unioned by this structure."""
        return iter(self.parents)

    def union(self, *objects):
        """Find the sets containing the objects and merge them all."""
        roots = [self[x] for x in objects]
        heaviest = max([(self.weights[r],r) for r in roots])[1]
        for r in roots:
            if r != heaviest:
                self.weights[heaviest] += self.weights[r]
                self.parents[r] = heaviest

In [10]:
from colormath.color_objects import LabColor, sRGBColor
from colormath.color_conversions import convert_color
from pyemd import emd
import itertools
dist = []
bins = 4
for a in itertools.product(range(4),repeat=3):
    for b in itertools.product(range(4),repeat=3):
        dist.append(np.linalg.norm(np.array(a)-np.array(b)))
dist = np.array(dist)
signatures = []
for sprite in id2colorized:
    image = id2colorized[sprite]
    grid = np.zeros([4,4,4])
    for yy in range(image.shape[0]):
        for xx in range(image.shape[1]):
            lab = convert_color(sRGBColor(image[yy,xx,0]/255.0,image[yy,xx,1]/255.0,image[yy,xx,2]/255.), LabColor)
            ind = np.array(lab.get_value_tuple())*float(bins)/100.0
            grid[int(ind[0]),int(ind[1]),int(ind[2])] += 1
    
    signature = []
    for a in itertools.product(range(4),repeat=3):
        signature.append(grid[a[0],a[1],a[2]])
    signatures.append(np.array(signature))

In [11]:
timesteps_bb = {}
for timestep in sorted_timesteps:
    dat = timesteps[timestep]
    if timestep% 1000 == 0:
        print timestep
    blobs = UnionFind()
    for ii in range(len(dat)):
        for jj in range(len(dat)):
            if ii != jj:
                id1 = dat[ii][0]
                dx = dat[ii][1]-dat[jj][1]
                dy = dat[ii][2]-dat[jj][2]

                id2 = (dat[jj][0], dx, dy)
                if (id1,id2) in accepted:
                    
                    blobs.union(ii,jj)
    merged = {}
    for ii in range(len(dat)):
        set_id = blobs[ii]
        if set_id not in merged:
            merged[set_id] = set()
        merged[set_id].add(ii)
    bounding_boxes = {}
    timesteps_bb[timestep] = bounding_boxes
    for set_id,sprites in merged.items():
        left = float('inf')
        right = float('-inf')
        top = float('inf')
        bottom = float('-inf')
        for sprite in sprites:
            left = min(dat[sprite][1],left)
            right = max(dat[sprite][1]+8,right)
            top = min(dat[sprite][2],top)
            bottom = max(dat[sprite][2]+8,bottom)
            
        bounding_boxes[ ((left+right)/2, (top+bottom)/2,left,right,top,bottom)] = merged[set_id]

1000
2000
3000
4000
5000
6000
7000


In [12]:
import networkx as nx
from networkx.algorithms import matching
import matplotlib.image as mpimg
import scipy.stats
def weight(data,track,R):
    distance = np.linalg.norm(data - track)
    return scipy.stats.norm(0, R).pdf(distance)

sigma = 8.0
min_gate = 3.0

tracks = {}
old_tracks = []
track_counter = 0
coast = {}

image_prefix = 'images/mario_{}.png'
output_prefix = 'tracks/mario_{}.png'

max_timestep = sorted_timesteps[-1]
min_timestep = sorted_timesteps[0]
for timestep in xrange(min_timestep,max_timestep+1):
    if timestep not in timesteps_bb:
        to_delete = set()
        for track in tracks:                
            if coast[track] > 4:
                old_tracks.append((track,tracks[track]))
                to_delete.add( track)
            else:
                coast[track] += 1                    
                tracks[track][timestep] = tracks[track][timestep-1]
    
        for track in to_delete:
            del tracks[track]
        continue
        
    bounding_boxes = timesteps_bb[timestep]
    locations = []
    B = nx.Graph()
    for track in tracks:
        
        B.add_node(track)
    for sprite_id,sprite in enumerate(bounding_boxes):
        B.add_node('sprite{}'.format(sprite_id))
        B.add_node('track_start{}'.format(sprite_id))
        B.add_edge('sprite{}'.format(sprite_id),'track_start{}'.format(sprite_id),
                   weight=scipy.stats.norm(0, sigma).pdf(min_gate*sigma))
        
        obs = np.array([sprite[0],240-sprite[1]])
               
        for track in tracks:
            pt = tracks[track][timestep-1][0]
            
            pt_weight = weight(obs,pt,8.0)
            B.add_edge('sprite{}'.format(sprite_id),track,
                   weight=pt_weight)
            
    match = matching.max_weight_matching(B)
    just_created = set()
    for sprite_id,sprite in enumerate(bounding_boxes):
        obs = np.array([sprite[0],240-sprite[1]])
        track = match['sprite{}'.format(sprite_id)]
        if 'start' in track:
            track_counter += 1
            just_created.add('track{}'.format(track_counter))
            tracks['track{}'.format(track_counter)] = {timestep:(obs,sprite,bounding_boxes[sprite])}
            coast['track{}'.format(track_counter)] = 0
            
        else:
            tracks[track][timestep] = (obs,sprite,bounding_boxes[sprite])
            coast[track] = 0
    to_delete = set()
    for track in tracks:
        if track not in match:
            if track not in just_created:
                if coast[track] > 4:
                    old_tracks.append((track,tracks[track]))
                    to_delete.add( track)
                else:
                    coast[track] += 1                    
                    tracks[track][timestep] = tracks[track][timestep-1]
    for track in to_delete:
        del tracks[track]
    if timestep %1000 == 0:
        print timestep
    if timestep % 10 == 0 and False:
        img=mpimg.imread(image_prefix.format(timestep))

        plt.imshow(img,extent=(0,256,0,240))
        for track in sorted(tracks):
            tr = []
            for ts in sorted(tracks[track]):
                tr.append(tracks[track][ts][0])
            tr = np.array(tr)
            plt.plot(tr[-100:,0],tr[-100:,1],'x-')
        plt.axis([0,256,0,240])
        plt.savefig(output_prefix.format(timestep), bbox_inches='tight')
        plt.clf()

1000
2000
3000
4000
5000
6000
7000


In [13]:

import pickle

pickle.dump((tracks,old_tracks),open('mario_tracks.pkl','wb'))